In [1]:
import os, sys
import spacy

from spacy.pipeline import EntityRuler

In [2]:
sys.path.append('..')

In [3]:
from cycontext.item_data import ItemData
from cycontext.tag_object import TagObject
from cycontext.context_component import ConTextComponent

In [4]:
%load_ext autoreload
%autoreload 2

In [5]:
nlp = spacy.load("en_core_web_sm")
_ = nlp.remove_pipe("ner")

In [6]:
# Use the EntityRuler class for rule-based NER
ruler = EntityRuler(nlp, overwrite_ents=True)

patterns = [
    [{"lower": "pneumonia"}],
    [{"lower": "asthma"}],
    [{"lower": "angina"}],
    [{"lower": "chf"}],
    [{"lower": "pe"}],
    [{"lower": {"REGEX": "clot"}}],
    [{"lower": "cancer"}],
]

patterns = [{"label": "CONDITION", "pattern": pattern} for pattern in patterns]

ruler.add_patterns(patterns)

In [7]:
try:
    _ = nlp.remove_pipe("entity_ruler")
except:
    pass
nlp.add_pipe(ruler)

In [8]:
item_data = [ItemData("no evidence of", category="DEFINITE_NEGATED_EXISTENCE", rule="forward"),
                  ItemData("but", category="TERMINATE", rule="terminate"),
            ItemData("is ruled out", category="DEFINITE_NEGATED_EXISTENCE", rule="backward"),
            ItemData("family history of", category="FAMILY_HISTORY", rule="forward")]

In [9]:
context = ConTextComponent(item_data, nlp)

In [10]:
if "context" in nlp.pipe_names:
    nlp.remove_pipe("context")
nlp.add_pipe(context)

In [11]:
sents = ["There is no evidence of pneumonia or asthma but there is angina.",
        "There is CHF.",
        "PE is ruled out.",
        "There is clotting but pe is ruled out.",
        "She has a family history of cancer."]

In [12]:
docs = list(nlp.pipe(sents))

In [13]:
docs

[There is no evidence of pneumonia or asthma but there is angina.,
 There is CHF.,
 PE is ruled out.,
 There is clotting but pe is ruled out.,
 She has a family history of cancer.]

In [14]:
for doc in docs:
    print(doc)
    for ent in doc.ents:
        print("\t- {0} modified by: {1}".format(ent, ent._.modifiers))

There is no evidence of pneumonia or asthma but there is angina.
	- pneumonia modified by: (<TagObject> [no evidence of, definite_negated_existence],)
	- asthma modified by: (<TagObject> [no evidence of, definite_negated_existence],)
	- angina modified by: ()
There is CHF.
	- CHF modified by: ()
PE is ruled out.
	- PE modified by: (<TagObject> [is ruled out, definite_negated_existence],)
There is clotting but pe is ruled out.
	- clotting modified by: ()
	- pe modified by: (<TagObject> [is ruled out, definite_negated_existence],)
She has a family history of cancer.
	- cancer modified by: (<TagObject> [family history of, family_history],)


# Visualization

In [15]:
from cycontext import viz

In [16]:
viz.visualize_targets(docs[-1])

In [17]:
viz.visualize_modifiers(docs[0])